In [24]:
import pandas as pd
from nltk import bigrams, ConditionalFreqDist
import re


In [28]:
file_path = './data/cleaned.csv'
data = pd.read_csv(file_path)
data['char_tokens'] = data['char_tokens'].apply(eval)


In [29]:
from nltk import ngrams
# Generate higher order n-grams
n = 3  # For trigrams
all_ngrams = []
for tokens in data['char_tokens']:
    token_ngrams = list(ngrams(tokens, n))
    all_ngrams.extend(token_ngrams)

# Update your frequency distribution to use these n-grams
cfd = ConditionalFreqDist((ngram[:-1], ngram[-1]) for ngram in all_ngrams)


# Function to apply diacritization
def diacritize(text):
    char_tokens = list(text)
    diacritized_text = []

    for i in range(len(char_tokens)):
        current_char = char_tokens[i]
        prev_char = char_tokens[i - 1] if i > 0 else None
        
        if current_char in ['c', 'g', 'i', 'o', 's', 'u']:  # Only check for characters that might need diacritics
            if prev_char:  # Check if there's a previous character to consider
                possible_chars = cfd[prev_char].most_common()
                if possible_chars:
                    # Find the most common successor that matches the potential diacritic versions of the current character
                    replacements = {
                        'c': 'ç', 'g': 'ğ', 'i': 'ı', 'o': 'ö', 's': 'ş', 'u': 'ü'
                    }
                    # Check if a replacement character exists in the predictions
                    predicted_chars = [p[0] for p in possible_chars]
                    for predicted_char in predicted_chars:
                        if predicted_char in replacements.values():
                            current_char = predicted_char
                            break
        diacritized_text.append(current_char)

    return ''.join(diacritized_text)


In [30]:
# Example usage
sample_text = "sisman bir cingene kadininin kucuk kizina bakmasi icin bir kucuk cocuk verilir."
diacritized_output = diacritize(sample_text)
print(diacritized_output)


sisman bir cingene kadininin kucuk kizina bakmasi icin bir kucuk cocuk verilir.
